In [1]:
import timm
import torchsummary
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from collections import OrderedDict
from timm.models import create_model, apply_test_time_pool, load_checkpoint, is_model, list_models, set_scriptable, set_no_jit
from timm.data import Dataset, DatasetTar, create_loader, resolve_data_config
from timm.utils import accuracy, AverageMeter, natural_key, setup_default_logging
from munch import Munch
import yaml
import sys
from efficientnet_pytorch import EfficientNet
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
import numpy as np
import os
from tqdm import tqdm

In [2]:
torch.backends.cudnn.benchmark = True

In [3]:
with open('config/save.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher

In [4]:
model_list = timm.list_models(pretrained=True)

In [5]:
model_list

['adv_inception_v3',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn107',
 'dpn131',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnetlight',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_es',
 'ens_adv_inception_resnet_v2',
 'ese_vovnet19b_dw',
 'ese_vovnet39b',
 'fbnetc_100',
 'gluon_inception_v3',
 'gluon_resnet18_v1b',
 'gluon_resnet34_v1b',
 'gluon_resnet50_v1b',
 'gluon_resnet50_v1c',
 'gluon_resnet50_v1d',
 'gluon_resnet50_v1s',
 'gluon_resnet101_v1b',
 'gluon_resnet101_v1c',
 'gluon_resnet101_v1d',
 'gluon_resnet101_v1s',
 'gluon_r

In [6]:
data_dir = '/home/data/imagenet/train'

In [7]:
# model_ns = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
# model_ns = model_ns.cuda()
# model = torch.nn.DataParallel(model_ns).cuda()

In [8]:
# model = EfficientNet.from_pretrained("efficientnet-b7", advprop=True)
# model_ap = model.cuda()
# model = torch.nn.DataParallel(model_ap).cuda()

In [9]:
model = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
model = model.cuda()
# model = torch.nn.DataParallel(model).cuda()

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
dataset = Dataset(data_dir, load_bytes=False, class_map='')

In [12]:
param_count = sum([m.numel() for m in model.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 7794184


In [13]:
data_config = resolve_data_config(args, model=model)
model, test_time_pool = apply_test_time_pool(model, data_config, args)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 240, 240)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.882


In [14]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
loader = create_loader(
    dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [15]:
filename_list = loader.dataset.filenames()

In [16]:
output_save = '/home/data/imagenet/save/output'
traj_save = '/home/data/imagenet/save/traj'

In [17]:
batch_time = AverageMeter()
losses = AverageMeter()
top1 = AverageMeter()
top5 = AverageMeter()

model.eval()
with torch.no_grad():
    batch_num = 0
    # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
    input = torch.randn((args.batch_size,) + data_config['input_size']).cuda()
    model(input)
    end = time.time()
    for i, (input, target) in tqdm(enumerate(loader)):
        if args.no_prefetcher:
            target = target.cuda()
            input = input.cuda()
        
        
        # compute output
        output, trajectory = model(input)
        output = output.detach().cpu().numpy()
        trajectory = trajectory.detach().cpu().numpy()
        
        for k in range(len(output)):
            output_name_list = filename_list[batch_num+k].split('/')
            output_name = output_name_list[-2] + '_' + os.path.splitext(output_name_list[-1])[0]
            
            np.save(os.path.join(output_save, output_name),output[[k]])
            np.save(os.path.join(traj_save, output_name), trajectory[[k]])
            
        batch_num += args.batch_size
        
#         loss = criterion(output, target)

#         # measure accuracy and record loss
#         acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
#         losses.update(loss.item(), input.size(0))
#         top1.update(acc1.item(), input.size(0))
#         top5.update(acc5.item(), input.size(0))

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()

#         if i % args.log_freq == 0:
#             logging.info(
#                 'Test: [{0:>4d}/{1}]  '
#                 'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
#                 'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
#                 'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
#                 'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
#                     i, len(loader), batch_time=batch_time,
#                     rate_avg=input.size(0) / batch_time.avg,
#                     loss=losses, top1=top1, top5=top5))

# results = OrderedDict(
#     top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
#     top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
#     param_count=round(param_count / 1e6, 2),
#     img_size=data_config['input_size'][-1],
#     cropt_pct=crop_pct,
#     interpolation=data_config['interpolation'])

# logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
#    results['top1'], results['top1_err'], results['top5'], results['top5_err']))


1117it [07:15,  2.35it/s]DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'sRGB' 41 1
DEBUG:PIL.PngImagePlugin:STREAM b'gAMA' 54 4
DEBUG:PIL.PngImagePlugin:STREAM b'cHRM' 70 32
DEBUG:PIL.PngImagePlugin:STREAM b'pHYs' 114 9
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 135 65401
3883it [31:37,  1.60it/s]/home/cutz/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
5005it [44:12,  1.89it/s]
